In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForMaskedLM
import plotly.io as pio
from umap import UMAP
import hdbscan

c:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from bertopic.representation import MaximalMarginalRelevance

Mostramos o diretório do arquivo CSV e definimos o dataframe utilizando a coluna de tweets limpos

In [131]:
caminho_arquivo = r'C:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\dados\dadosTwitter\CSV\resitencia_bacteria\tweets_processados.csv'
tweets_df = pd.read_csv(caminho_arquivo)
tweets_df = tweets_df[tweets_df["language"] == "pt"]


In [132]:
tweets_df.dropna(axis='columns', inplace=True)
tweets = tweets_df["tokens"].tolist()

### Usar dados do G1

In [63]:
caminho_arquivo = r'C:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\dados\dadosG1\bacteria\news_processados.csv'
tweets_df = pd.read_csv(caminho_arquivo)
tweets_df.dropna(axis='columns', inplace=True)
tweets = tweets_df["tokens"].tolist()

# BERTimbau Base para Topic Modeling

Primeiro definimos o modelo de embedding, no caso, o BERTimbau

In [56]:
embedding_model = SentenceTransformer("neuralmind/bert-base-portuguese-cased")

No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.


# Configurando o UMAP
técnica de redução de dimensionalidade. Ele pega seus embeddings de alta dimensão e os projeta em um espaço de menor dimensão enquanto tenta preservar a estrutura local e global dos dados

- n_neighbors: Controla o balanço entre a preservação da estrutura local versus global dos dados.
    Valores menores (por exemplo, 2 a 10) fazem com que o UMAP se concentre mais na estrutura local dos dados, resultando em clusters mais apertados e potencialmente mais tópicos, mas que podem ser muito específicos.
    Valores maiores (por exemplo, 15 a 50 ou mais) fazem com que o UMAP preserve mais a estrutura global, resultando em clusters mais dispersos e potencialmente menos tópicos, mas mais genéricos.
    Um bom ponto de partida é geralmente entre 10 e 15.
- n_components: A dimensão para a qual os embeddings serão reduzidos.
    Para visualização, geralmente usamos n_components=2 ou 3.
    Para o agrupamento, usar n_components entre 5 e 10 é comum. Um número maior de componentes pode preservar mais informações dos embeddings, mas pode tornar o agrupamento mais difícil para o HDBSCAN se houver muito "ruído".
- metric: A métrica de distância usada para calcular a proximidade entre os pontos nos embeddings.
    'cosine' é uma escolha comum e geralmente boa para embeddings de texto, pois mede a similaridade angular entre os vetores, o que é relevante para o significado semântico.



Treinamos os modelos nos Tweets já tokenizados

In [ ]:
umap_model = UMAP(n_neighbors=10, #5 para mais clusters
                  n_components=5, 
                  metric='cosine',
                  random_state=42
                 )

# Configurando o HDBSCAN
É um algoritmo de agrupamento que agrupa pontos de dados com base na densidade. Ele é excelente para encontrar clusters de várias formas e tamanhos, e também consegue identificar "ruído" (documentos que não pertencem a nenhum tópico).

- min_cluster_size: O número mínimo de amostras em um cluster para ser considerado um cluster válido.
    Valores menores resultam em mais clusters e potencialmente mais ruído (tópicos com poucos documentos).
    Valores maiores resultam em menos clusters, mas mais densos e potencialmente mais robustos.

- cluster_selection_method: Define como os clusters são selecionados a partir da hierarquia.
    'eom' (default, "Excess of Mass") é geralmente a melhor opção, pois tende a produzir clusters mais equilibrados.
    'leaf' seleciona os clusters nas folhas da hierarquia, o que pode resultar em muitos clusters pequenos.

- metric='euclidean

In [165]:
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=7,
                                min_samples=5,
                                metric='euclidean',
                                cluster_selection_method='eom',
                                prediction_data=True 
                               )

# Inicializar o BERTopic com os modelos configurados e o parâmetro diversity

In [167]:
representation_model = MaximalMarginalRelevance(diversity=0.1, top_n_words=10)

topic_model = BERTopic(embedding_model=embedding_model,
                       language="portuguese",
                       umap_model=umap_model,
                       hdbscan_model=hdbscan_model,
                       representation_model=representation_model,
                       nr_topics="auto", # Ou um número fixo, ex: 50
                       min_topic_size=10 # Mantendo o padrão ou ajustando
                      )


Treinar o modelo

In [168]:
topics, probs = topic_model.fit_transform(tweets)

In [169]:
tweets_df["topic"] = topics
tweets_df["topic_prob"] = probs


In [170]:
topic_model.get_topic(1) # Exibir os tópicos encontrados
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,397,-1_bactéria_antibiótico_infecção_medicamento,"[bactéria, antibiótico, infecção, medicamento,...",[criar resistência remédio mau existir ser bac...
1,0,505,0_antibiótico_bactéria_tomar_vírus,"[antibiótico, bactéria, tomar, vírus, médico, ...",[acontecer demora tratar resistente bactéria f...
2,1,50,1_estudo_bovino_associar_bactéria,"[estudo, bovino, associar, bactéria, molecular...",[faculdade ciência farmacêutico ribeirão preto...
3,2,42,2_klebsiella_pneumoniae_gonorreia_bactério,"[klebsiella, pneumoniae, gonorreia, bactério, ...",[resistência antibiótico bactéria estômago pyl...
4,3,32,3_brasil_rolar_aura_petista,"[brasil, rolar, aura, petista, podre, haraway,...",[entender perder laranja podre conseguir barra...
5,4,23,4_antibiotico_bacteria_resistencia_resistencer,"[antibiotico, bacteria, resistencia, resistenc...",[caso certo tomar outro antibiotico bacteria c...
6,5,22,5_vcs_virulêncer_mdsss_rafaelluiz,"[vcs, virulêncer, mdsss, rafaelluiz, nelsoncez...",[rafaelluiz estudo apontar informação resistên...
7,6,21,6_fico_enhum_hauhauaha_vsf,"[fico, enhum, hauhauaha, vsf, huauhuhauhauhauh...",[parar cross parar comigo mto feliz resistênci...
8,7,19,7_clavulanato_clavunato_ampicilinar_cefalotino,"[clavulanato, clavunato, ampicilinar, cefaloti...",[certo remédio tirar fator resistência imagine...
9,8,14,8_una_venezuela_quien_eso,"[una, venezuela, quien, eso, mucho, narco, nac...",[cuar entra una propiedad privado tiene derech...


In [172]:
tweets_df
tweets_df.to_excel(r'C:\\Users\\Pessoal\\Documents\\Unifesp\Aries\\aries_topic_selector\\dados\\processados\\tweets_processados_com_bert_resitencia_bacteria.xlsx', index=False)

In [171]:
topic_model.visualize_barchart(top_n_topics=10)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.01199822493450686, 0.014612791369684816,
                    0.01804566946228622, 0.022277965881095645,
                    0.024404685078979676],
              'xaxis': 'x',
              'y': [médico  , vírus  , tomar  , bactéria  , antibiótico  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.016356632998167992, 0.016613803127240225,
                    0.017251681483943064, 0.021808843997557326,
                    0.030851982745524247],
              'xaxis': 'x2',
              'y': [molecular  , bactéria  , associar  , bovino  , estudo  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03406976883015221, 0.03564067256454183, 0.03969014868606341,
                    0.05885076529610206, 0.06191488364213835],
              'xaxis': 'x3',
              'y': [gonorrhoeae  , bactério  , gonorreia  , pneumoniae  ,
                    klebsiella  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.02040966547343611, 0.02040966547343611, 0.02040966547343611,
                    0.02551209799658597, 0.0278657322352756],
              'xaxis': 'x4',
              'y': [podre  , petista  , aura  , rolar  , brasil  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04646807517145173, 0.07563776487514418, 0.09784901864047801,
                    0.125199148538717, 0.13488185981719253],
              'xaxis': 'x5',
              'y': [virus  , resistencer  , resistencia  , bacteria  , antibiotico
                    ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03436090932092894, 0.03436090932092894, 0.03436090932092894,
                    0.03436090932092894, 0.042165422621369214],
              'xaxis': 'x6',
              'y': [nelsoncezar  , rafaelluiz  , mdsss  , virulêncer  , vcs  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.025734739616930038, 0.025734739616930038,
                    0.025734739616930038, 0.025734739616930038,
                    0.033224200113406877],
              'xaxis': 'x7',
              'y': [huauhuhauhauhauhahuo  , vsf  , hauhauaha  , enhum  , fico  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.030737056567466916, 0.03318896969084595,
                    0.03318896969084595, 0.03318896969084595, 0.0384565774010273],
              'xaxis': 'x8',
              'y': [serratir  , cefalotino  , ampicilinar  , clavunato  ,
                    clavulanato  ],
              'yaxis': 'y8'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.056576015690613046, 0.056576015690613046,
                    0.056576015690613046, 0.056576015690613046,
                    0.17598055917882333],
              'xaxis': 'x9',
              'y': [mucho  , eso  , quien  , venezuela  , una  ],
              'yaxis': 'y9'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04489491071858598, 0.052978446763955744,
                    0.07970197830867386, 0.15391987791895356, 0.215487829086535],
              'xaxis': 'x10',
             

In [45]:
topic_model.save("meu_modelo_bertopic")

2025-06-17 16:11:04,680 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
